In [19]:
import os
import json
import math
import pandas as pd
from datetime import date, time
import numpy as np
import psycopg2

In [20]:
# Establecer la semilla aleatoria para reproducibilidad
np.random.seed(123)

In [21]:
data = pd.read_csv('../datasets/processed/yellow_analytics/yellow_analytics_part_1.csv', sep=',', encoding='utf8')

In [22]:
data.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,DuracionViaje
0,1,2,3.80,1.0,142,236,1,21.95,2022-01-01,00:35:40,2022-01-01,00:53:29,1069
1,1,1,2.10,1.0,236,42,1,13.30,2022-01-01,00:33:43,2022-01-01,00:42:07,504
2,2,1,1.09,1.0,114,68,2,11.80,2022-01-01,00:25:21,2022-01-01,00:35:23,602
3,2,1,4.30,1.0,68,163,1,30.30,2022-01-01,00:36:48,2022-01-01,01:14:20,2252
4,2,1,5.07,1.0,233,87,1,26.00,2022-01-01,00:20:50,2022-01-01,00:34:58,848


In [23]:
data.shape

(364131, 13)

In [7]:
data.IdProveedor.unique()

array([1, 2], dtype=int64)

In [12]:
data.IdProveedor.value_counts()

IdProveedor
2    256555
1    107576
Name: count, dtype: int64

In [24]:
#Calcular el tamaño de muestra proporcional para cada proveedor
proporciones = data['IdProveedor'].value_counts(normalize=True) * 0.12  # Proporción deseada
tamaño_muestra_por_proveedor = (proporciones * len(data)).astype(int)

# Lista para almacenar las muestras aleatorias estratificadas
muestras_estratificadas = []

# Iterar sobre cada proveedor único
for proveedor in data['IdProveedor'].unique():
    # Seleccionar aleatoriamente el tamaño de muestra proporcional para el proveedor actual
    muestra_proveedor = data[data['IdProveedor'] == proveedor].sample(tamaño_muestra_por_proveedor[proveedor])
    # Agregar las muestras seleccionadas al DataFrame de muestras estratificadas
    muestras_estratificadas.append(muestra_proveedor)

# Concatenar las muestras estratificadas en un solo DataFrame
muestras_estratificadas = pd.concat(muestras_estratificadas)


In [25]:
muestras_estratificadas.shape

(43695, 13)

In [17]:
muestras_estratificadas.IdProveedor.value_counts()

IdProveedor
2    30786
1    12909
Name: count, dtype: int64

In [26]:
muestras_estratificadas.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,DuracionViaje
157178,1,1,2.1,1.0,238,48,1,14.55,2022-01-05,08:12:18,2022-01-05,08:23:52,694
166028,1,1,2.5,1.0,68,229,1,17.75,2022-01-05,12:04:07,2022-01-05,12:19:35,928
262756,1,1,1.0,1.0,107,164,1,12.35,2022-01-07,16:32:32,2022-01-07,16:38:07,335
317951,1,3,2.3,1.0,161,246,1,14.80,2022-01-08,18:24:19,2022-01-08,18:39:52,933
289658,1,2,5.5,1.0,114,238,1,27.35,2022-01-08,02:29:39,2022-01-08,02:50:26,1247


In [32]:
muestras_estratificadas.dtypes


IdProveedor         int64
TotalPasajeros      int64
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida       object
FechaLlegada       object
HoraLlegada        object
DuracionViaje       int64
dtype: object

In [28]:
# Conectar a la base de datos PostgreSQL
conn = psycopg2.connect(
    dbname="nycdb",
    user="postgres",
    password="mailito",
    host="localhost"
)


In [29]:
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

In [30]:
# Crear la tabla en la base de datos
cur.execute("""
    CREATE TABLE registros (
        Id SERIAL PRIMARY KEY,
        IdProveedor INTEGER,
        TotalPasajeros INTEGER,
        DistanciaViaje FLOAT,
        IdTipoTarifa FLOAT,
        IdZonaOrigen INTEGER,
        IdZonaDestino INTEGER,
        IdTipoPago INTEGER,
        CostoTotal FLOAT,
        FechaRecogida DATE,
        HoraRecogida TIME,
        FechaLlegada DATE,
        HoraLlegada TIME,
        DuracionViaje INTEGER
    )
""")

# Guardar los cambios en la base de datos
conn.commit()

In [34]:
# Convertir los valores del DataFrame a listas de Python
registros = muestras_estratificadas.values.tolist()

# Insertar los registros en la tabla
cur.executemany("""
    INSERT INTO registros (
        IdProveedor, TotalPasajeros, DistanciaViaje, IdTipoTarifa, IdZonaOrigen, IdZonaDestino, IdTipoPago,
        CostoTotal, FechaRecogida, HoraRecogida, FechaLlegada, HoraLlegada, DuracionViaje
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", registros)

# Guardar los cambios en la base de datos
conn.commit()

# Cerrar la conexión con la base de datos
cur.close()
conn.close()

print("Registros migrados exitosamente a PostgreSQL.")

Registros migrados exitosamente a PostgreSQL.


In [1]:
import site
import airflow

# Obtener la ruta de instalación de airflow
airflow_path = airflow.__path__[0]

# Obtener las rutas de los paquetes de Python instalados
python_paths = site.getsitepackages()

print("Ruta de instalación de Airflow:", airflow_path)
print("Rutas de paquetes de Python instalados:")
for path in python_paths:
    print(path)

c:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\airflow\configuration.py:812 DeprecationWarning: The sql_alchemy_conn option in [core] has been moved to the sql_alchemy_conn option in [database] - the old setting has been used, but please update your config.

c:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\airflow\configuration.py:738 DeprecationWarning: The sql_alchemy_conn option in [core] has been moved to the sql_alchemy_conn option in [database] - the old setting has been used, but please update your config.

c:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\airflow\settings.py:194 DeprecationWarning: The sql_alchemy_conn option in [core] has been moved to the sql_alchemy_conn option in [database] - the old setting has been used, but please update your config.

Ruta de instalación de Airflow: c:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\airflow
Rutas de paquetes de Python instalados:
c:\Users\ozi\AppData\Local\Programs\Python\Python310
c:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages
